In [1]:
import json
#import _cvxcore
#import cvxpy
import fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,SimpleFill
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from datetime import timedelta
from datetime import datetime

In [2]:
activity=pd.read_csv('.../aaai/data/air/initial/AirQualityUCI.csv',header=0)

In [3]:
activity.head()

,Date,Time,Month,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,3,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,3,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,3,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,3,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,3,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [44]:
activity.isna().sum()

Date             0
Time             0
Month            0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64

In [45]:
activity.shape

(9357, 16)

In [4]:
rows = activity.groupby('Month').count()['Date']%20

In [5]:
rows

Month
3     10
4      0
5      4
6      0
7      4
8      4
9      0
10     4
11     0
12     4
13     4
14    12
15     4
16     7
Name: Date, dtype: int64

In [6]:
rows=pd.DataFrame(rows)

In [7]:
rows=rows.reset_index()

In [8]:
final=pd.DataFrame()
for seq in range(rows.shape[0]):
    #print(rows.iloc[seq,0])
    #print(rows.iloc[seq,1])
    
    temp=activity[activity['Month']==rows.iloc[seq,0]]
    #print(temp.shape)
    nrows = temp.shape[0]-rows.iloc[seq,1]
    temp=temp.iloc[0:nrows,:]
    #print(temp.shape)
    #print(temp.shape[0]%20)
    final=pd.concat([final,temp])

In [9]:
final.shape

(9300, 16)

In [10]:
final.groupby('Month').count()['Date']%20

Month
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
Name: Date, dtype: int64

In [53]:
#print(maskTrain['interval'].isna().sum())
#print(activityTrain['Time'].isna().sum())

In [11]:
final.head()

,Date,Time,Month,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,3,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,3,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,3,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,3,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,3,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [13]:
final.groupby('Month').count()['Time']

Month
3     500
4     720
5     740
6     720
7     740
8     740
9     720
10    740
11    720
12    740
13    740
14    660
15    740
16     80
Name: Time, dtype: int64

In [23]:
(final[final['CO(GT)']==-200].count())

Date             1679
Time             1679
Month            1679
CO(GT)           1679
PT08.S1(CO)      1679
NMHC(GT)         1679
C6H6(GT)         1679
PT08.S2(NMHC)    1679
NOx(GT)          1679
PT08.S3(NOx)     1679
NO2(GT)          1679
PT08.S4(NO2)     1679
PT08.S5(O3)      1679
T                1679
RH               1679
AH               1679
dtype: int64

In [22]:
final['CO(GT)'].shape

(9300,)

In [24]:
1679/9300

0.18053763440860215

In [20]:
9300*13

120900

In [21]:
#missing%
16642/120900

0.1376509511993383

In [137]:
activityTrain=final[final['Month'].isin([3,4,6,8,9,11,12,13,15,16])]
activityVal=final[final['Month'].isin([5])]
activityTest=final[final['Month'].isin([7,10,14])]

In [138]:
#maskTrain = pd.DataFrame(np.ones((activityTrain.shape[0], activityTrain.shape[1])),columns=activityTrain.columns)
#maskTrain['Time']=activityTrain['Time']

In [139]:
maskTrain=activityTrain[['CO(GT)','Date','Time']].copy()
maskTrain[maskTrain['CO(GT)']!=-200]=1
maskTrain[maskTrain['CO(GT)']==-200]=0
maskTrain[['Date','Time']]=activityTrain[['Date','Time']].copy()

In [140]:
maskVal=activityVal[['CO(GT)','Date','Time']].copy()
maskVal[maskVal['CO(GT)']!=-200]=1
maskVal[maskVal['CO(GT)']==-200]=0
maskVal[['Date','Time']]=activityVal[['Date','Time']].copy()

In [141]:
maskTest=activityTest[['CO(GT)','Date','Time']].copy()
maskTest[maskTest['CO(GT)']!=-200]=1
maskTest[maskTest['CO(GT)']==-200]=0
maskTest[['Date','Time']]=activityTest[['Date','Time']].copy()

In [142]:
maskVal.head()

,CO(GT),Date,Time
1230,1.0,5/1/2004,0:00:00
1231,1.0,5/1/2004,1:00:00
1232,1.0,5/1/2004,2:00:00
1233,1.0,5/1/2004,3:00:00
1234,0.0,5/1/2004,4:00:00


In [143]:
print(activityTest['CO(GT)'].isna().sum())

0


In [68]:
activityTest=activityTest.fillna(0)

In [128]:
maskTrain.head()

,CO(GT),Date,Time
0,1.0,3/10/2004,18:00:00
1,1.0,3/10/2004,19:00:00
2,1.0,3/10/2004,20:00:00
3,1.0,3/10/2004,21:00:00
4,1.0,3/10/2004,22:00:00


In [144]:
#decay
def decay(data):
    print(data.head())
    data['interval']=0
    #df=data.groupby('person_id')
    j=0
    for n in range(int(data.shape[0]/20)):
        i=0
        #print(n)
        df_group=data.iloc[n*20:(n*20)+20,:]
        for index, row in df_group.iterrows():#go over mask
            if(i==0):
                row['interval']=0
                i=1
            else:
                if(prev['CO(GT)']==1):
                    row['interval']=timedelta.total_seconds(datetime.strptime(str(row['Date'])+" "+str(row['Time']),"%m/%d/%Y %H:%M:%S")-datetime.strptime(str(prev['Date'])+" "+str(prev['Time']),"%m/%d/%Y %H:%M:%S"))
                elif(prev['CO(GT)']==0):
                    row['interval']=timedelta.total_seconds(datetime.strptime(str(row['Date'])+" "+str(row['Time']),"%m/%d/%Y %H:%M:%S")-datetime.strptime(str(prev['Date'])+" "+str(prev['Time']),"%m/%d/%Y %H:%M:%S"))+prev['interval']
            prev=row
            data.iloc[j,3]=row['interval']
            j=j+1

    data['interval'] = data['interval'].apply(lambda x: abs(x/60))
    print(data.head())
    return data

In [145]:
def rdecay(data):
    print(data.head())
    data['intervalReverse']=0
    j=data.shape[0]-1
    for n in range(int(data.shape[0]/20)):
        i=0
        df_group=data.iloc[n*20:(n*20)+20,:]
        df_group=df_group[::-1]
        for index, row in df_group.iterrows():#go over mask
            if(i==0):
                row['intervalReverse']=0
                i=1
            else:
                if(prev['CO(GT)']==1):
                    row['intervalReverse']=timedelta.total_seconds(datetime.strptime(str(row['Date'])+" "+str(row['Time']),"%m/%d/%Y %H:%M:%S")-datetime.strptime(str(prev['Date'])+" "+str(prev['Time']),"%m/%d/%Y %H:%M:%S"))
                elif(prev['CO(GT)']==0):
                    row['intervalReverse']=timedelta.total_seconds(datetime.strptime(str(row['Date'])+" "+str(row['Time']),"%m/%d/%Y %H:%M:%S")-datetime.strptime(str(prev['Date'])+" "+str(prev['Time']),"%m/%d/%Y %H:%M:%S"))+prev['interval']
            prev=row
            data.iloc[j,4]=row['intervalReverse']
            j=j-1

    data['intervalReverse'] = data['intervalReverse'].apply(lambda x: abs(x/60))
    print(data.head())
    return data

In [146]:
maskTrain=decay(maskTrain)
print(maskTrain.head())
maskTrain=rdecay(maskTrain)

   CO(GT)       Date      Time
0     1.0  3/10/2004  18:00:00
1     1.0  3/10/2004  19:00:00
2     1.0  3/10/2004  20:00:00
3     1.0  3/10/2004  21:00:00
4     1.0  3/10/2004  22:00:00
   CO(GT)       Date      Time  interval
0     1.0  3/10/2004  18:00:00       0.0
1     1.0  3/10/2004  19:00:00      60.0
2     1.0  3/10/2004  20:00:00      60.0
3     1.0  3/10/2004  21:00:00      60.0
4     1.0  3/10/2004  22:00:00      60.0
   CO(GT)       Date      Time  interval
0     1.0  3/10/2004  18:00:00       0.0
1     1.0  3/10/2004  19:00:00      60.0
2     1.0  3/10/2004  20:00:00      60.0
3     1.0  3/10/2004  21:00:00      60.0
4     1.0  3/10/2004  22:00:00      60.0
   CO(GT)       Date      Time  interval
0     1.0  3/10/2004  18:00:00       0.0
1     1.0  3/10/2004  19:00:00      60.0
2     1.0  3/10/2004  20:00:00      60.0
3     1.0  3/10/2004  21:00:00      60.0
4     1.0  3/10/2004  22:00:00      60.0
   CO(GT)       Date      Time  interval  intervalReverse
0     1.0  3/10/20

In [147]:
maskVal=decay(maskVal)
print(maskVal.head())
maskVal=rdecay(maskVal)

      CO(GT)      Date     Time
1230     1.0  5/1/2004  0:00:00
1231     1.0  5/1/2004  1:00:00
1232     1.0  5/1/2004  2:00:00
1233     1.0  5/1/2004  3:00:00
1234     0.0  5/1/2004  4:00:00
      CO(GT)      Date     Time  interval
1230     1.0  5/1/2004  0:00:00       0.0
1231     1.0  5/1/2004  1:00:00      60.0
1232     1.0  5/1/2004  2:00:00      60.0
1233     1.0  5/1/2004  3:00:00      60.0
1234     0.0  5/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval
1230     1.0  5/1/2004  0:00:00       0.0
1231     1.0  5/1/2004  1:00:00      60.0
1232     1.0  5/1/2004  2:00:00      60.0
1233     1.0  5/1/2004  3:00:00      60.0
1234     0.0  5/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval
1230     1.0  5/1/2004  0:00:00       0.0
1231     1.0  5/1/2004  1:00:00      60.0
1232     1.0  5/1/2004  2:00:00      60.0
1233     1.0  5/1/2004  3:00:00      60.0
1234     0.0  5/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval  intervalR

In [148]:
maskTest=decay(maskTest)
print(maskTest.head())
maskTest=rdecay(maskTest)

      CO(GT)      Date     Time
2694     1.0  7/1/2004  0:00:00
2695     1.0  7/1/2004  1:00:00
2696     1.0  7/1/2004  2:00:00
2697     1.0  7/1/2004  3:00:00
2698     1.0  7/1/2004  4:00:00
      CO(GT)      Date     Time  interval
2694     1.0  7/1/2004  0:00:00       0.0
2695     1.0  7/1/2004  1:00:00      60.0
2696     1.0  7/1/2004  2:00:00      60.0
2697     1.0  7/1/2004  3:00:00      60.0
2698     1.0  7/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval
2694     1.0  7/1/2004  0:00:00       0.0
2695     1.0  7/1/2004  1:00:00      60.0
2696     1.0  7/1/2004  2:00:00      60.0
2697     1.0  7/1/2004  3:00:00      60.0
2698     1.0  7/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval
2694     1.0  7/1/2004  0:00:00       0.0
2695     1.0  7/1/2004  1:00:00      60.0
2696     1.0  7/1/2004  2:00:00      60.0
2697     1.0  7/1/2004  3:00:00      60.0
2698     1.0  7/1/2004  4:00:00      60.0
      CO(GT)      Date     Time  interval  intervalR

In [149]:
activityTrain.to_csv('.../aaai/data/mimic/preprocess/airTrain.csv',index=False)
maskTrain.to_csv('.../aaai/data/mimic/preprocess/airTrainMask.csv',index=False)

In [150]:
activityVal.to_csv('.../aaai/data/mimic/preprocess/airVal.csv',index=False)
maskVal.to_csv('.../aaai/data/mimic/preprocess/airValMask.csv',index=False)

In [151]:
activityTest.to_csv('.../aaai/data/mimic/preprocess/airTest.csv',index=False)
maskTest.to_csv('.../aaai/data/mimic/preprocess/airTestMask.csv',index=False)